In [53]:
%matplotlib inline
import pandas as pd
import numpy as np
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))# nrows=1048576)

In [54]:
data.head()

,CountryCode,IndicatorCode,Year,Value
0,ARB,SP.ADO.TFRT,1960,1.335609e+02
1,ARB,SP.POP.DPND,1960,8.779760e+01
2,ARB,SP.POP.DPND.OL,1960,6.634579e+00
3,ARB,SP.POP.DPND.YG,1960,8.102333e+01
4,ARB,MS.MIL.XPRT.KD,1960,3.000000e+06


In [55]:
columns=['CountryCode']
countries = pd.read_csv("data/Country.csv", usecols=(columns))

In [56]:
countries.head()

,CountryCode
0,AFG
1,ALB
2,DZA
3,ASM
4,ADO


In [57]:
# find some measures that seem important and are often present, filter countries with all those intact, train network

In [58]:
columns=['LongDefinition', 'SeriesCode']
series = pd.read_csv("data/Series.csv", usecols=(columns))

In [59]:
series.head()

,SeriesCode,LongDefinition
0,BN.KLT.DINV.CD,Foreign direct investment are the net inflows ...
1,BX.KLT.DINV.WD.GD.ZS,Foreign direct investment are the net inflows ...
2,BX.KLT.DINV.CD.WD,Foreign direct investment refers to direct inv...
3,BM.KLT.DINV.GD.ZS,Foreign direct investment are the net inflows ...
4,BN.TRF.KOGT.CD,Net capital account records acquisitions and d...


In [60]:
print (len(countries))
print (len(series))

247
1345


In [61]:
#isUsedSeries = [(name in seriesToUse) for name in series.SeriesCode.values]
#filteredSeries = series[isUsedSeries]
#filteredSeriesNames = filteredSeries.SeriesCode.values

#useableData = [(name in filteredSeriesNames) for name in data.IndicatorCode.values]
#filteredData = data[useableData]
seriesCounts = data['IndicatorCode'].value_counts()

a = seriesCounts.to_dict()
useableData = [(a.get(name, 0) > 10000) for name in data.IndicatorCode.values]
filteredData = data[useableData]

In [62]:
useableSeries = [(a.get(name, 0) > 10000) for name in series.SeriesCode.values]
filteredSeries = series[useableSeries].SeriesCode.values

In [63]:
print(len(filteredData))
print(len(data))
print(len(filteredSeries))
print(len(series))

732948
5656458
63
1345


In [66]:
# a = data[(data.CountryCode == 'ARB') & ((data.IndicatorCode == 'SP.ADO.TFRT') | (data.IndicatorCode == 'SP.POP.DPND'))]
useableCountries = []
for country in countries.CountryCode.values:
    useable = True
    filterByCountry = filteredData[(data.CountryCode == country)]
    for seriesName in filteredSeries:
        if len(filterByCountry[(filterByCountry.IndicatorCode == seriesName)]) == 0:
            useable = False;
            break;
    if useable:
        useableCountries.append(country)

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [67]:
len(useableCountries)

175

In [68]:
goodData = [(country in useableCountries) for country in filteredData.CountryCode.values]
useableData = filteredData[goodData]

In [69]:
print(len(useableData))
useableData.head()

559891


,CountryCode,IndicatorCode,Year,Value
350,EAP,SP.ADO.TFRT,1960,75.043631
351,EAP,SP.POP.DPND,1960,78.574185
352,EAP,SP.POP.DPND.OL,1960,6.480808
353,EAP,SP.POP.DPND.YG,1960,71.976235
359,EAP,SP.DYN.CBRT.IN,1960,26.635950


In [70]:
# Filtered Data reasonably above, need to:
# Seperate into few year blocks, with next year values as results to guess
# Train a few different models to guess next years values

In [ ]:
predictionFeature = 'SP.ADO.TFRT'

trainingValues = []
trainingResults = []
for country in useableCountries[0:40]: # 213
    countryData = filteredData[(filteredData.CountryCode == country)]
    pastData_5 = []
    pastData_4 = []
    pastData_3 = []
    pastData_2 = []
    pastData_1 = []
    for year in range(1960, 2010): # 1960
        pastData_5 = pastData_4
        pastData_4 = pastData_3
        pastData_3 = pastData_2
        pastData_2 = pastData_1
        
        yearData = countryData[(countryData.Year == year)]
        
        pastData_1 = []
        for series in filteredSeries[0:30]: # 40
            # TODO: see if I can get rid of this loop
            data = yearData[(yearData.IndicatorCode == series)].Value.values
            if len(data) != 1:
                pastData_5 = []
                pastData_4 = []
                pastData_3 = []
                pastData_2 = []
                pastData_1 = []
                break;
            value = data[0]
            #print (value)
            pastData_1.append(value)
            
        predictionValue = countryData[(countryData.Year == year + 1) & (countryData.IndicatorCode == predictionFeature)].Value.values
        valid = (len(predictionValue) == 1) & (len(pastData_5) != 0) & (len(pastData_4) != 0) & (len(pastData_3) != 0) & (len(pastData_2) != 0) & (len(pastData_1) != 0)
        if valid:
            trainingValues.append(pastData_5 + pastData_4 + pastData_3 + pastData_2 + pastData_1)
            trainingResults.append(predictionValue[0])

In [ ]:
xData = pd.DataFrame(trainingValues)
yData = pd.DataFrame(trainingResults)
xData

In [36]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(xData, yData)

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.

In [27]:
a = [1, 2]
b = [3, 4]
d = [6, 4]
c = a+b+d
c

[1, 2, 3, 4, 6, 4]

In [48]:
useableCountries

['AFG',
 'ALB',
 'DZA',
 'AGO',
 'ATG',
 'ARG',
 'ARM',
 'AUS',
 'AUT',
 'AZE',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BOL',
 'BIH',
 'BRA',
 'BRN',
 'BGR',
 'BFA',
 'BDI',
 'CPV',
 'KHM',
 'CMR',
 'CAN',
 'CAF',
 'TCD',
 'CHL',
 'CHN',
 'COL',
 'COM',
 'COG',
 'CRI',
 'CIV',
 'HRV',
 'CUB',
 'CYP',
 'CZE',
 'ZAR',
 'DNK',
 'DOM',
 'EAP',
 'ECU',
 'EGY',
 'SLV',
 'EST',
 'ETH',
 'ECA',
 'FJI',
 'FIN',
 'FRA',
 'GAB',
 'GEO',
 'DEU',
 'GHA',
 'GRC',
 'GRD',
 'GTM',
 'GIN',
 'GNB',
 'GUY',
 'HTI',
 'HIC',
 'HND',
 'HUN',
 'IND',
 'IDN',
 'IRN',
 'IRQ',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KOR',
 'KWT',
 'KGZ',
 'LAO',
 'LAC',
 'LVA',
 'LBN',
 'LBR',
 'LBY',
 'LTU',
 'LMY',
 'LMC',
 'LUX',
 'MKD',
 'MDG',
 'MWI',
 'MYS',
 'MDV',
 'MLI',
 'MLT',
 'MRT',
 'MUS',
 'MEX',
 'MNA',
 'MIC',
 'MDA',
 'MNG',
 'MNE',
 'MAR',
 'MOZ',
 'MMR',
 'NPL',
 'NLD',
 'NZL',
 'NIC',
 'NER',
 'NGA',
 'NOR',
 'OMN',
 'PAK',
 'PAN',
 'PNG',
 'PRY',
 'PER',
 'PHL',
 'POL',


In [ ]:
for country in useableCountries[0:15]: # 213
    countryData = filteredData[(filteredData.CountryCode == country)]
    for startYear in range(1960, 2000):
        
        
        
        row = []
        for additionalYear in range(0, 5):
            yearData = countryData[(countryData.Year == startYear + additionalYear)]
            for series in filteredSeries[0:15]: # 40
                data = yearData[(yearData.IndicatorCode == series)].Value.values
                if len(data) != 1:
                    valid = False
                    break;
                value = data[0]
                row.append(value)
        
        predictionValue = countryData[(countryData.Year == startYear + 6) & (countryData.IndicatorCode == predictionFeature)].Value.values
        if len(predictionValue) != 1:
            valid = False
        if valid:
            trainingValues.append(row)
            trainingResults.append(predictionValue[0])